In [1]:
import numpy as np # numerical calc package
import pandas as pd # holds data
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # pretty plotting
import pandas_profiling as pp

sns.set(rc={'figure.figsize':(20,10)})

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split # split dataset
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor as xgb

from sklearn.metrics import mean_squared_error as mse # Measurement metric
from math import sqrt

In [2]:
#importing the data set
data = pd.read_csv('data/nyc-rolling-sales.csv')

#### Data Preparation

In [3]:
data.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [4]:
data.shape

(84548, 22)

In [5]:
data.describe()

,Unnamed: 0,BOROUGH,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
count,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000,84548.000000
mean,10344.359878,2.998758,4237.218976,376.224015,10731.991614,2.025264,0.193559,2.249184,1789.322976,1.657485
std,7151.779436,1.289790,3568.263407,658.136814,1290.879147,16.721037,8.713183,18.972584,537.344993,0.819341
min,4.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4231.000000,2.000000,1322.750000,22.000000,10305.000000,0.000000,0.000000,1.000000,1920.000000,1.000000
50%,8942.000000,3.000000,3311.000000,50.000000,11209.000000,1.000000,0.000000,1.000000,1940.000000,2.000000
75%,15987.250000,4.000000,6281.000000,1001.000000,11357.000000,2.000000,0.000000,2.000000,1965.000000,2.000000
max,26739.000000,5.000000,16322.000000,9106.000000,11694.000000,1844.000000,2261.000000,2261.000000,2017.000000,4.000000


In [6]:
data.dtypes

Unnamed: 0                         int64
BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
EASE-MENT                         object
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                        object
SALE DATE                         object
dtype: object

In [7]:
# Creating Profile Report
# report=pp.ProfileReport(data)

In [8]:
#transforming data types to proper format
data['SALE PRICE']=pd.to_numeric(data['SALE PRICE'],errors='coerce')
data['LAND SQUARE FEET']=pd.to_numeric(data['LAND SQUARE FEET'],errors='coerce')
data['GROSS SQUARE FEET']=pd.to_numeric(data['GROSS SQUARE FEET'],errors='coerce')

data['SALE DATE'] = pd.to_datetime(data['SALE DATE'], errors='coerce')


In [9]:
#checking for missing data
data.isna().sum()

Unnamed: 0                            0
BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING CLASS CATEGORY               0
TAX CLASS AT PRESENT                  0
BLOCK                                 0
LOT                                   0
EASE-MENT                             0
BUILDING CLASS AT PRESENT             0
ADDRESS                               0
APARTMENT NUMBER                      0
ZIP CODE                              0
RESIDENTIAL UNITS                     0
COMMERCIAL UNITS                      0
TOTAL UNITS                           0
LAND SQUARE FEET                  26252
GROSS SQUARE FEET                 27612
YEAR BUILT                            0
TAX CLASS AT TIME OF SALE             0
BUILDING CLASS AT TIME OF SALE        0
SALE PRICE                        14561
SALE DATE                             0
dtype: int64

In [10]:
to_impute = data[['LAND SQUARE FEET', 'GROSS SQUARE FEET', 'SALE PRICE']]

In [11]:
imp = SimpleImputer(strategy='median')
imp.fit(to_impute)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [12]:
#imputing data with the media
imputed_data = imp.transform(to_impute.values)
imputed_data_df = pd.DataFrame(imputed_data, columns=to_impute.columns)
imputed_data_df.head()

,LAND SQUARE FEET,GROSS SQUARE FEET,SALE PRICE
0,1633.0,6440.0,6625000.0
1,4616.0,18690.0,530000.0
2,2212.0,7803.0,530000.0
3,2272.0,6794.0,3936272.0
4,2369.0,4615.0,8000000.0


In [13]:
new_data = data.combine_first(imputed_data_df)
data = new_data
data.isna().sum()

ADDRESS                           0
APARTMENT NUMBER                  0
BLOCK                             0
BOROUGH                           0
BUILDING CLASS AT PRESENT         0
BUILDING CLASS AT TIME OF SALE    0
BUILDING CLASS CATEGORY           0
COMMERCIAL UNITS                  0
EASE-MENT                         0
GROSS SQUARE FEET                 0
LAND SQUARE FEET                  0
LOT                               0
NEIGHBORHOOD                      0
RESIDENTIAL UNITS                 0
SALE DATE                         0
SALE PRICE                        0
TAX CLASS AT PRESENT              0
TAX CLASS AT TIME OF SALE         0
TOTAL UNITS                       0
Unnamed: 0                        0
YEAR BUILT                        0
ZIP CODE                          0
dtype: int64

In [14]:
data.shape

(84548, 22)

In [15]:
zero = 0
fifth = data['SALE PRICE'].describe(np.arange(0.05, 1, 0.05)).T['15%']
ninetyfifth = data['SALE PRICE'].describe(np.arange(0.05, 1, 0.05)).T['95%']
data = data[(data['SALE PRICE'] > zero) &
            (data['SALE PRICE'] <= ninetyfifth)].copy()

In [16]:
data.shape

(70163, 22)

In [17]:
data.dtypes

ADDRESS                                   object
APARTMENT NUMBER                          object
BLOCK                                      int64
BOROUGH                                    int64
BUILDING CLASS AT PRESENT                 object
BUILDING CLASS AT TIME OF SALE            object
BUILDING CLASS CATEGORY                   object
COMMERCIAL UNITS                           int64
EASE-MENT                                 object
GROSS SQUARE FEET                        float64
LAND SQUARE FEET                         float64
LOT                                        int64
NEIGHBORHOOD                              object
RESIDENTIAL UNITS                          int64
SALE DATE                         datetime64[ns]
SALE PRICE                               float64
TAX CLASS AT PRESENT                      object
TAX CLASS AT TIME OF SALE                  int64
TOTAL UNITS                                int64
Unnamed: 0                                 int64
YEAR BUILT          

In [18]:
dep = 'SALE PRICE'
cat = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT',
              'BUILDING CLASS AT PRESENT', 'BUILDING CLASS AT TIME OF SALE']
cont = ['LAND SQUARE FEET', 'GROSS SQUARE FEET', 'RESIDENTIAL UNITS', 
              'COMMERCIAL UNITS']


In [19]:
data.shape

(70163, 22)

In [20]:
# write list of features we want to use
X=['BOROUGH', 'BLOCK', 'LOT', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
      'TAX CLASS AT TIME OF SALE']
X=transformed_newdata[X]
y = transformed_newdata["SALE PRICE"]


NameError: name 'transformed_newdata' is not defined

In [ ]:
X.columns

In [ ]:
sns.heatmap(X.corr(),square=True,cmap='coolwarm')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100)

In [ ]:
# initialize model
linear_reg = LinearRegression()

# fit model
linear_reg.fit(X_train, y_train)

# metrics (accuracy)
#acc = linear_reg.score(X_test, y_test)
#COeffecient
coefficient= linear_reg.coef_
#GET INTERCEPTS
intercepts=linear_reg.intercept_
# predictions
y_pred = linear_reg.predict(X_test)

In [ ]:
# feature importance
coefficient

In [ ]:
intercepts

In [ ]:
rmse = np.sqrt(mse(y_test, y_pred))

print(rmse)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, 
max_depth = 5, alpha = 10, n_estimators = 10)


In [ ]:
rmse = np.sqrt(mse(y_test, y_pred))

print(rmse)
